In [ ]:
import pandas as pd
import os

def import_data():
    # Percorsi ai file JSON
    base_paths = [
        "data/automated-v1.0",
        "data/fake-v1.0"
    ]

    files = {
        "automated": "automatedAccountData.json",
        "nonautomated": "nonautomatedAccountData.json",
        "fake": "fakeAccountData.json",
        "real": "realAccountData.json"
    }

    # Costruisci i percorsi completi
    paths = {
        "automated": os.path.join(base_paths[0], files["automated"]),
        "nonautomated": os.path.join(base_paths[0], files["nonautomated"]),
        "fake": os.path.join(base_paths[1], files["fake"]),
        "real": os.path.join(base_paths[1], files["real"]),
    }

    # Leggi i file JSON come DataFrame
    df_automated = pd.read_json(paths["automated"])
    df_nonautomated = pd.read_json(paths["nonautomated"])
    df_fake = pd.read_json(paths["fake"])
    df_real = pd.read_json(paths["real"])

    # Unisci tutti i DataFrame in uno solo
    df_automated = pd.concat([df_automated, df_nonautomated], ignore_index=True)
    df_fake = pd.concat([df_fake, df_real], ignore_index=True)

    # Mostra un’anteprima
    df_automated.head()
    df_fake.head()
    #print(f"\nTotale righe: {len(df_all)}")
    return df_automated, df_fake

In [14]:
df_automated, df_fake = import_data()

   userMediaCount                                   mediaLikeNumbers  \
0              97  [100, 78, 112, 78, 77, 111, 97, 95, 63, 83, 42...   
1             863  [229, 331, 180, 326, 313, 157, 245, 161, 182, ...   
2             149  [237, 254, 350, 257, 180, 172, 141, 84, 218, 8...   
3              33  [163, 30, 92, 111, 145, 110, 81, 81, 61, 60, 6...   
4            1000  [61, 222, 437, 829, 681, 440, 920, 256, 1017, ...   

                                 mediaCommentNumbers  \
0  [5, 2, 10, 0, 3, 2, 2, 2, 4, 0, 0, 6, 6, 0, 2,...   
1  [3, 2, 3, 4, 7, 2, 0, 6, 2, 2, 1, 9, 2, 0, 2, ...   
2  [12, 16, 37, 11, 3, 4, 2, 4, 5, 1, 1, 2, 3, 3,...   
3  [2, 2, 7, 14, 17, 3, 2, 5, 2, 1, 5, 5, 8, 1, 4...   
4  [4, 0, 13, 9, 9, 6, 16, 4, 25, 6, 3, 4, 16, 10...   

                            mediaCommentsAreDisabled  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0,